## This file is the first version of processing video frames, and it's deprecated. See store_frames_new.ipynb

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone

load_dotenv()
client = OpenAI() # api_key=

pc = Pinecone() # api_key=
index = pc.Index("ta")

Capture the frame for each 10 secondes. For each frame, use OCR to recognize the text and save them in a json file.

In [3]:
import json
import os
import cv2
import os
import pytesseract
from PIL import Image
import datetime

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extract_text_from_image(image):
    image_pil = Image.fromarray(image)
    text = pytesseract.image_to_string(image_pil)
    
    return text

videos=[]
folder_path='./videos'
files=os.listdir(folder_path)

frame_texts={}
for file in files[:17]: 
    frame_text={}
    path=folder_path+'/'+file
    # Open the video file
    video = cv2.VideoCapture(path)

    # Get the total number of frames in the video
    # total_frames = min(int(video.get(cv2.CAP_PROP_FRAME_COUNT)),100)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the interval between frames (every 10 seconds)
    frame_interval_sec = 10
    frame_rate = video.get(cv2.CAP_PROP_FPS)
    frame_interval = int(frame_interval_sec * frame_rate)

    # Set the current frame to the first frame in the video
    current_frame = 0

    # Loop through the frames and extract frames at the desired interval
    for i in range(total_frames):
        # Set the video's current frame to the desired frame
        video.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        
        # Read the frame
        ret, frame = video.read()
        
        if frame is None:
            break

        # Extract text from the frame
        text = extract_text_from_image(frame)
        if text:
            # Get the timestamp of the frame
            timestamp_ms = video.get(cv2.CAP_PROP_POS_MSEC)
            timestamp_sec = timestamp_ms / 1000.0
            timestamp_str = str(datetime.timedelta(seconds=timestamp_sec)).split('.')[0]#.replace(':', '_')
            
            # Save the frame to filematch_test/
            # file_name = f'./frame_{i}_{timestamp_str}'
            # cv2.imwrite(file_name+'.jpg', frame)
            frame_text[timestamp_str]=text

        # Move to the next frame according to the interval
        current_frame += frame_interval
    
    frame_texts[file]=frame_text

    # Release the video file
    video.release()

with open('frame_text.json', "w") as json_file:
    json.dump(frame_texts, json_file)      

Read the above json file and do the embedding, upsert the vectors into Pinecone.

In [13]:
import json
with open('./frame_text.json', "r") as json_file:
    frames = json.load(json_file)

for file_name, frame in frames.items():
    for time_s, frame_content in frame.items():
        res = client.embeddings.create(input=[frame_content], model="text-embedding-3-small")
        vector=res.data[0].embedding
        index.upsert(
            vectors=[
                {
                "id": f"Video-{file_name.replace('【','').replace('】 ','-')}-{time_s}", 
                "values": vector, 
                "metadata": {"file_name": f"{file_name}", "time": time_s}
                }
            ],
            namespace='video_data')